# Tag the Tweet!

Tagging a Tweet based on its context! Using Recurrent Neural Networks, NGrams.

# Problem Statement

        Tag the Tweet is an Application built on python, which understands all the tweets of the trending hashtags
        and predicts a hashtag for the given tweets.

In [53]:
import numpy as np
import sys
import re, datetime, pandas as pd
import tweepy as tw
from sklearn.utils import shuffle

%matplotlib inline
#from ResultMiner import TweetMiner

# Step - 01

We need a lot of data regarding the Tweets! Hence I am using `tweepy` to authenticate my Twitter Keys and use the API to retrieve the required data.

In [8]:
MAX_HASHTAGS_COUNT = 3
MIN_HASHTAGS_COUNT = 2
US_WOE_ID = 23424977
INDIA_WOE_ID = 23424848
UK_WOE_ID = 23424975
TWEET_LIMIT = 125

twitter_details = {
    'consumer_key'        : "pNPFulq7iU1AwsH8WKXcM3VhZ",
    'consumer_secret'     : "k6weUOaZxYSbBBLCBB5DsshL8iUDWIOLY5WlHK555kHRWOinuQ",
    'access_token_key'    : "782632596-Zx3pED7JikyG7dBx1a6oIqyQh60qeSMUXHAqjfhg",
    'access_token_secret' : "DoqjnFjyJIqEEsqUVtnyWX2KYyswA9rTCPtJr9GGfyTHL"
}

auth = tw.OAuthHandler(twitter_details['consumer_key'], twitter_details['consumer_secret'])
auth.set_access_token(twitter_details['access_token_key'], twitter_details['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

In [9]:
class TweetMiner():

    def __init__(self, api, tweet_limit):
        
        self.api = api
        self.tweet_limit = tweet_limit
        
        
    def retrieve_data(self, tag, WOE_ID, page_limit):
        
        cleaned_data = []
        latest_max_id = False
        jk = True
        for i in range(page_limit):
            tweets = tw.Cursor(self.api.search,
                     q=tag+" -filter:retweets",
                     lang="en",
                     max_id = latest_max_id,
                     since=2018-1-1).items(self.tweet_limit)
            
            for tweet in tweets:
                if latest_max_id != tweet.id:
                    try:
                        data = {
                            'id' : tweet.id,
                            'user_name' : tweet.user.screen_name,
                            'tweet_description' : tweet.text,
                            'retweet_count' : tweet.retweet_count,
                            'created_at' : tweet.created_at,
                            'Location_ID' : WOE_ID,
                            'hashtag' : tag
                        }
                    except:
                        data = {
                            'id' : tweet.id,
                            'user_name' : tweet.user.screen_name,
                            'tweet_description' : tweet.text,
                            'retweet_count' : 0,
                            'created_at' : tweet.created_at,
                            'Location_ID' : WOE_ID,
                            'hashtag' : tag
                        }
                    latest_max_id = tweet.id
                    cleaned_data.append(data)
        
        return cleaned_data

In [10]:
tweet_miner = TweetMiner(api, TWEET_LIMIT)

In [11]:
def get_trending_hashtags(trends, hashtags):
    trends = trends['trends']
    trending_hashtags = []
    for trend in trends:
        print(trend['name'] + str(trend['tweet_volume']))
        print("_____________")
        if trend['name'][0] == '#' and trend['tweet_volume'] != None and trend['tweet_volume'] >= 2500:
            if not trend in hashtags:
                trending_hashtags = trend
                break
    return trending_hashtags

In [12]:
hashtags = []
hashtags.append(get_trending_hashtags(api.trends_place(US_WOE_ID)[0], hashtags))
hashtags.append(get_trending_hashtags(api.trends_place(INDIA_WOE_ID)[0], hashtags))
hashtags.append(get_trending_hashtags(api.trends_place(UK_WOE_ID)[0], hashtags))

#CollegeInSixWordsNone
_____________
Julio Jones46329
_____________
#PanVisibilityDay29840
_____________
#TwitterIndia34775
_____________
KillieNone
_____________
#CorrieNone
_____________
DundeeNone
_____________
#BTSxTheOneShow44265
_____________


# Tweet Retrievel from Trending HashTags
  We are trying to understand three popular hashtags by mining 1500 tweets each..
  Below are the pandas dataframes for each of these. Which we later try to merge back
  as on dataset X

In [13]:
print(hashtags[0])
print("_________________________")
print(hashtags[1])
print("_________________________")
print(hashtags[2])
print("_________________________")
print(hashtags)
data = []

{'name': '#PanVisibilityDay', 'url': 'http://twitter.com/search?q=%23PanVisibilityDay', 'promoted_content': None, 'query': '%23PanVisibilityDay', 'tweet_volume': 29840}
_________________________
{'name': '#TwitterIndia', 'url': 'http://twitter.com/search?q=%23TwitterIndia', 'promoted_content': None, 'query': '%23TwitterIndia', 'tweet_volume': 34775}
_________________________
{'name': '#BTSxTheOneShow', 'url': 'http://twitter.com/search?q=%23BTSxTheOneShow', 'promoted_content': None, 'query': '%23BTSxTheOneShow', 'tweet_volume': 44265}
_________________________
[{'name': '#PanVisibilityDay', 'url': 'http://twitter.com/search?q=%23PanVisibilityDay', 'promoted_content': None, 'query': '%23PanVisibilityDay', 'tweet_volume': 29840}, {'name': '#TwitterIndia', 'url': 'http://twitter.com/search?q=%23TwitterIndia', 'promoted_content': None, 'query': '%23TwitterIndia', 'tweet_volume': 34775}, {'name': '#BTSxTheOneShow', 'url': 'http://twitter.com/search?q=%23BTSxTheOneShow', 'promoted_content': 

In [14]:
data.append(tweet_miner.retrieve_data(hashtags[0]['name'], US_WOE_ID, 20))
US_df = pd.DataFrame(data[0])

In [15]:
US_df

,id,user_name,tweet_description,retweet_count,created_at,Location_ID,hashtag
0,1396923606773469189,wormyHQ,happy #PanVisibilityDay !!! remember every sin...,0,2021-05-24 20:18:36,23424977,#PanVisibilityDay
1,1396923591988420609,JordanWoollenVO,Oh is it #PanVisibilityDay ? I'm still trying ...,0,2021-05-24 20:18:32,23424977,#PanVisibilityDay
2,1396923589606191109,JobsSummary,New job Plant Front End Loader Operator -Dotha...,0,2021-05-24 20:18:32,23424977,#PanVisibilityDay
3,1396923540306337798,soulmatesbts,"i’m out here (just annoying everyone on sc, le...",0,2021-05-24 20:18:20,23424977,#PanVisibilityDay
4,1396923518454018048,rogeliovall123,Happy #PanVisibilityDay,0,2021-05-24 20:18:15,23424977,#PanVisibilityDay
...,...,...,...,...,...,...,...
2476,1396862489028083713,_0ptical,"WOO, HAPPY #PanVisibilityDay !!! PAN ARTISTS/C...",24,2021-05-24 16:15:44,23424977,#PanVisibilityDay
2477,1396862433185214464,TEENAGEWEBHEADD,happy #PanVisibilityDay to me and peter maximo...,0,2021-05-24 16:15:31,23424977,#PanVisibilityDay
2478,1396862368303419398,StpCalinMeThicc,happy #PanVisibilityDay 🥳🥳💕💕💕💕💕,0,2021-05-24 16:15:15,23424977,#PanVisibilityDay
2479,1396862336313479170,SkjeidyBrady,Happy #PanVisibilityDay to any and all celebra...,0,2021-05-24 16:15:08,23424977,#PanVisibilityDay


In [16]:
data.append(tweet_miner.retrieve_data(hashtags[1]['name'], INDIA_WOE_ID, 20))
India_df = pd.DataFrame(data[1])

In [17]:
India_df

,id,user_name,tweet_description,retweet_count,created_at,Location_ID,hashtag
0,1396927080621740033,KritiPandey06,@sakshijoshii chamchas talking against India w...,0,2021-05-24 20:32:24,23424848,#TwitterIndia
1,1396927003022921728,Wild_Monk_,The bird(murderer of democracy) must be caged....,0,2021-05-24 20:32:05,23424848,#TwitterIndia
2,1396926862828347393,Ramkuma78515142,. . . good to know the priority of the governm...,0,2021-05-24 20:31:32,23424848,#TwitterIndia
3,1396926823636762624,Vikrantkumarrr,#TwitterIndiaRaid #TwitterIndia \nIn next two ...,0,2021-05-24 20:31:23,23424848,#TwitterIndia
4,1396926590781583363,anantjain14,#TwitterIndia stay strong. \n#TwitterIndiaRaid,0,2021-05-24 20:30:27,23424848,#TwitterIndia
...,...,...,...,...,...,...,...
2476,1396867768583028736,NaveenS92216228,*Twitter India office raided by Delhi Police ...,0,2021-05-24 16:36:43,23424848,#TwitterIndia
2477,1396867767752527877,ind_transformer,#twitterindia\nMr. Trump now. https://t.co/ktU...,0,2021-05-24 16:36:43,23424848,#TwitterIndia
2478,1396867767169474564,Sikande15419419,#TwitterIndia\n\n@TwitterIndia it for u. Take ...,1,2021-05-24 16:36:42,23424848,#TwitterIndia
2479,1396867766167035909,Gandipolitics5,Now who did this 😂😂\n#AmitShah #KanganaRanaut ...,0,2021-05-24 16:36:42,23424848,#TwitterIndia


In [18]:
data.append(tweet_miner.retrieve_data(hashtags[2]['name'], UK_WOE_ID, 20))
UK_df = pd.DataFrame(data[2])

In [19]:
UK_df

,id,user_name,tweet_description,retweet_count,created_at,Location_ID,hashtag
0,1396929775432785925,MariMoraes3,@BBCTheOneShow @BTSPurpleBullet @BBCiPlayer @B...,0,2021-05-24 20:43:06,23424975,#BTSxTheOneShow
1,1396929214063026176,ttinygoo,my mum watched #BTSxTheOneShow interview with ...,0,2021-05-24 20:40:52,23424975,#BTSxTheOneShow
2,1396928778891415552,IlovesmusicBTS,#BTSxTheOneShow\n I'm so happy and grateful BT...,0,2021-05-24 20:39:09,23424975,#BTSxTheOneShow
3,1396928510065664000,BBC_travelogue,"@BleacherReport @brgridiron @undisputed Hi, I ...",0,2021-05-24 20:38:05,23424975,#BTSxTheOneShow
4,1396928254578069504,BBC_travelogue,"@barstoolsports Hi, I am a professional #digit...",0,2021-05-24 20:37:04,23424975,#BTSxTheOneShow
...,...,...,...,...,...,...,...
2162,1396760382920871937,FundsForBTS_UK,BTS will be on The One Show TODAY on @BBCOne!\...,456,2021-05-24 09:30:00,23424975,#BTSxTheOneShow
2163,1396428993608437760,pengtingseokjin,Tomorrow we got The One Show. Remember to enga...,6,2021-05-23 11:33:11,23424975,#BTSxTheOneShow
2164,1395894226060288003,BTSRM_UK,So Ronan Keating @ronanofficial our Irish silk...,1,2021-05-22 00:08:12,23424975,#BTSxTheOneShow
2165,1395389993372192770,GirlWithLuv24,BTS will be appearing on BBC One's 'The One Sh...,10,2021-05-20 14:44:34,23424975,#BTSxTheOneShow


# Ngrams

Ngrams helps us in understanding intresting things about the corpus.

## For example 
We can know the similarity of each tweet under a specific tag using the Ngrams. 
For this purpose we shall:

1) 

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from collections import Counter

vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(US_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('panvisibilityday https', 353),
 ('happy panvisibilityday', 340),
 ('visibility day', 221),
 ('pan visibility', 114),
 ('panvisibilityday pan', 105),
 ('pan visibility day', 105),
 ('panvisibilityday fellow', 101),
 ('panvisibilityday happy', 99),
 ('pansexual panromantic', 89),
 ('panvisibilityday pansexual', 82),
 ('panvisibilitydayhappy panvisibilityday', 81),
 ('panvisibilityday happy panvisibilityday', 63),
 ('day panvisibilityday', 61),
 ('fellow pansexuals', 58),
 ('pansexual visibility', 58),
 ('pansexual visibility day', 57),
 ('panvisibilityday love', 56),
 ('happy pan', 55),
 ('panvisibilityday lt', 47),
 ('panvisibilityday valid', 47)]

In [21]:
vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(India_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('twitter india', 329),
 ('twitterindia https', 327),
 ('delhi police', 278),
 ('_क यर_ह', 256),
 ('india office', 192),
 ('twitter india office', 188),
 ('raid twitter', 154),
 ('special cell', 153),
 ('raid twitter india', 138),
 ('police special', 136),
 ('police special cell', 135),
 ('twitterindia office', 125),
 ('raid twitter india office', 121),
 ('twitterindia twitterindia', 113),
 ('twitterindia _क', 101),
 ('twitterindia _क यर_ह', 101),
 ('twitterindia twitterraid', 86),
 ('यर_ह twitterindia', 85),
 ('_क यर_ह twitterindia', 85),
 ('यर_ह https', 79)]

In [22]:
vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(UK_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('btsxtheoneshow btsonbbc', 236),
 ('btsxtheoneshow https', 183),
 ('bts btsxtheoneshow', 68),
 ('bts_twt https', 65),
 ('btsonbbc https', 62),
 ('btsonbbc bts_twt', 59),
 ('bts_twt bbctheoneshow', 59),
 ('make happy', 58),
 ('btsxtheoneshow btsonbbc https', 58),
 ('btsxtheoneshow btsonbbc bts_twt', 58),
 ('happy charge', 55),
 ('charge batteries', 55),
 ('btsxtheoneshow bbctheoneshow', 55),
 ('btsonbbc btsxtheoneshow', 54),
 ('btsxtheoneshow bts_twt', 54),
 ('bbctheoneshow bts_twt', 53),
 ('make happy charge', 52),
 ('try make', 49),
 ('bbctheoneshow btsxtheoneshow', 48),
 ('team try', 46)]

# Preprocessing and Feature Extraction

Now, we need to concat the data we collected from three of these hashtags. We need to prepare our input data and output data to feed them to our model, before doing that we need to:

1) Clean the data, by which I mean removing the URLs, emails, punctuations, etc.

2) Then we need to build a features vector using TF-IDF.

In [23]:
tweets_data = pd.concat([US_df, India_df, UK_df], axis =0)
tweets_data = shuffle(tweets_data)
tweets_data.to_csv('hastags_dataframes.csv')

We use textacy.preprocessing for this module to preprocess the data.

## Note: 
If there is an error installing your textacy saying unable to build wheels for scikit, it has something do with C-compiler in your system.

In [24]:
from bs4 import BeautifulSoup
from textacy.preprocessing import replace
from textacy.preprocessing import remove
from textacy.preprocessing.normalize import normalize_whitespace

In [25]:
def preprocess_tweets(tweet_descriptions):
    processed_tweet = []
    for tweet in tweet_descriptions:
        #BeautifulSoup is the fastest in my experience and hence using it for parsing the HTML
        tweet = BeautifulSoup(tweet,'html.parser').get_text() #HTML Parser
        
        #Replace is also imported from textacy.preprocessing where I wanted to replace certain data with 
        # generalized terms. Instead of removing them by assigning the parameter replace_with = '', I chose a category
        
        tweet = replace.replace_emails(tweet,replace_with= 'EMAIL')
        tweet = replace.replace_urls(tweet,replace_with= 'URL')
        tweet = replace.replace_currency_symbols(tweet,replace_with= 'CURRENCY')
        tweet = replace.replace_phone_numbers(tweet,replace_with= 'PHONE')
        tweet = replace.replace_emojis(tweet, replace_with='')
        #Below are a few methods defined under 'remove' from textacy.preprocessing
        #We use it remove Accent characters, punctuations, etc.
        tweet = remove.remove_accents(tweet)
        tweet = remove.remove_punctuation(tweet)
        
        tweet = re.sub(' +', ' ', tweet) #Removal of Whitespace; Using regEx to improve the performance
        processed_tweet.append(tweet)
    return processed_tweet

In [26]:
tweet_desc = tweets_data['tweet_description'].values
print(tweet_desc[1:4])

['#TwitterIndia office is being raided because it called fake what is actually fake. \n\nWhy can’t Sambit Patra prove h… https://t.co/kJUuYF6jYq'
 'Not gonna lie,when i heard "what\'s your favorite food?" question i was like "armys?really?are armys really asking o… https://t.co/7xP16m6tnP'
 'Tracy beaker omg HI?!?   #BTSxTheOneShow']


In [27]:
processed_tweets = preprocess_tweets(tweet_desc)
print(processed_tweets[1:4])

[' TwitterIndia office is being raided because it called fake what is actually fake \n\nWhy can t Sambit Patra prove h URL', 'Not gonna lie when i heard what s your favorite food question i was like armys really are armys really asking o URL', 'Tracy beaker omg HI BTSxTheOneShow']


# Prepare your Dataset

Now that the preprocessing of data is finished, unwanted noises are removed we can prepare our dataset (X, Y)

## Input

The processed_tweets is our input dataset, however, before feeding to our model there is one last thing to perform that is feature extraction. 

1) Feature extraction is done using TF-IDF (Term Frequency - Inverse Document Frequency).
2) The dimensionality reduction helps us in reducing our processing time without altering the meaning and accuracy.

## Output

This is a multi-classification problem, hence

1) we have three different classes each representing a trending hashtag from three different countries.
2) We run the data through a loop and set the y value of that particular tweet to 0 = US, 1 = India, 2 = UK.

Note: Before preparing the dataset remember to shuffle the data. So that there are no complications going further.

## Train & Test Dataset

Divide the data into train and test dataset, inorder to have equal distribution we perform a shuffle before.

In [39]:
Y = []

for tag in tweets_data['hashtag'].values:
    if tag == hashtags[0]['name']:
        Y.append(0)
    if tag == hashtags[1]['name']:
        Y.append(1)
    if tag == hashtags[2]['name']:
        Y.append(2)

X, Y = shuffle(processed_tweets, Y)
X_train_tweet = X[:7100]
X_test_tweet =X[7101:]

Y_train = Y[:7100]
Y_test = Y[7101:]

In [49]:
tf_idf = tfidf(ngram_range=(2,5), max_features=1500)
X_train = tf_idf.fit_transform(X_train_tweet).todense()
X_test = tf_idf.transform(X_test_tweet).todense()
#Important point here is to understand why we use fit_transform for train data & transform for test data

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(multi_class='multinomial')
params = {'penalty': ['l2'], 'C':np.logspace(-5,0,100)}

gs = GridSearchCV(lr, param_grid=params, cv=10, verbose=1)
gs.fit(X_train, Y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 28.8min finished


GridSearchCV(cv=10, estimator=LogisticRegression(multi_class='multinomial'),
             param_grid={'C': array([1.00000000e-05, 1.12332403e-05, 1.26185688e-05, 1.41747416e-05,
       1.59228279e-05, 1.78864953e-05, 2.00923300e-05, 2.25701972e-05,
       2.53536449e-05, 2.84803587e-05, 3.19926714e-05, 3.59381366e-05,
       4.03701726e-05, 4.53487851e-05, 5.09413801e-05, 5.72236766e-05,
       6.42807312e-...
       6.89261210e-02, 7.74263683e-02, 8.69749003e-02, 9.77009957e-02,
       1.09749877e-01, 1.23284674e-01, 1.38488637e-01, 1.55567614e-01,
       1.74752840e-01, 1.96304065e-01, 2.20513074e-01, 2.47707636e-01,
       2.78255940e-01, 3.12571585e-01, 3.51119173e-01, 3.94420606e-01,
       4.43062146e-01, 4.97702356e-01, 5.59081018e-01, 6.28029144e-01,
       7.05480231e-01, 7.92482898e-01, 8.90215085e-01, 1.00000000e+00]),
                         'penalty': ['l2']},
             verbose=1)

In [45]:
print(gs.best_params_)
print(gs.best_score_*100)

{'C': 0.7924828983539169, 'penalty': 'l2'}
89.0281690140845


That's a pretty decent score. However, we shall proceed with other models like:
>
    * Random forest
    * KNN
    * SVM

In [48]:
test_tweets = pd.DataFrame(X_test_tweet, columns=["Tweet_description"])
predict_tweet = pd.DataFrame(gs.predict_proba(X_test),columns=["US", "IND", "UK"])
pd.merge(test_tweets, predict_tweet, left_index=True, right_index=True)

,Tweet_description,US,IND,UK
0,Delhi police searching TwitterIndia office for...,0.004165,0.991383,0.004452
1,Delhi police special cell went to raid twitter...,0.014601,0.970197,0.015202
2,What a great introduction from the hosts \n Th...,0.149942,0.273486,0.576572
3,I AM SO EXCITED TO SEE OUR BOYS BTSxTheOneShow,0.037178,0.047643,0.915179
4,Happy PanVisibilityDay to all of you lovely Pa...,0.976030,0.011349,0.012621
5,ARMY s HAPPINESS AND TEAM \n\n BTSxTheOneShow URL,0.020523,0.025803,0.953674
6,DelhiPolice raids Twitter office in connectio...,0.078890,0.830164,0.090946
7,happy PanVisibilityDay,0.999753,0.000120,0.000127
8,Happy PanVisibilityDay \n uhmmm hi hellooo I m...,0.999753,0.000120,0.000127
9,Delhi police special cell went to raid twitter...,0.014601,0.970197,0.015202


`Note`: In this project, we have maintained seperate training data and test data only for our understanding.
Without which we can also use cross_val_score from sklearn.

In [68]:
from sklearn import svm

svm_clsf = svm.SVC(kernel='linear')

svm_clsf.fit(X_train, Y_train)

SVC(kernel='linear')

In [71]:
print(svm_clsf.score(X_test, Y_test)*100)

96.42857142857143


96% is fantastic!

In [78]:
Y_SVM_pred = svm_clsf.predict(X_test)
predict_country = []
for pred in Y_SVM_pred:
    if pred == 0:
        predict_country.append("US")
    elif pred == 1:
        predict_country.append("India")
    else:
        predict_country.append("UK")
SVM_result = pd.merge(test_tweets, pd.DataFrame(predict_country, columns=["Country"]), left_index=True, right_index=True)       

In [79]:
SVM_result

,Tweet_description,Country
0,Delhi police searching TwitterIndia office for...,India
1,Delhi police special cell went to raid twitter...,India
2,What a great introduction from the hosts \n Th...,UK
3,I AM SO EXCITED TO SEE OUR BOYS BTSxTheOneShow,UK
4,Happy PanVisibilityDay to all of you lovely Pa...,US
5,ARMY s HAPPINESS AND TEAM \n\n BTSxTheOneShow URL,UK
6,DelhiPolice raids Twitter office in connectio...,India
7,happy PanVisibilityDay,US
8,Happy PanVisibilityDay \n uhmmm hi hellooo I m...,US
9,Delhi police special cell went to raid twitter...,India
